<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/process/demo_field_process_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demo Field process model (standalone)

Define input parameters

# Process Simulation Workbook

This workbook demonstrates the simulation of a multi-stage oil and gas separation and gas injection process using the NeqSim process simulator.

The simulation models a typical upstream oil and gas facility, including:
- **Well Stream Input**: Hydrocarbon and water feed based on specified compositions and flow rates.
- **Separation Train**: Three stages of separation (1st, 2nd, and 3rd stage separators) to separate oil, gas, and water.
- **Gas Compression**: Multi-stage compression with inter-stage cooling and scrubbing to boost gas pressure.
- **Gas Injection**: Distribution of compressed gas to multiple injection trains, each with its own compression stages.

The key objective of this simulation is to analyze and optimize the process performance under various operating conditions and design cases.

## Input Parameters

Input parameters for the simulation are defined in the `ProcessInput` dataclass and can be configured through an Excel file (`process_conditions.xlsx`) for different years or design scenarios. Well stream compositions and flow rates are also loaded from Excel files.

## NeqSim Library

NeqSim (Norwegian Equation of State Simulator) is an open-source process simulator developed at the Norwegian University of Science and Technology (NTNU). It is based on advanced thermodynamic models and is widely used for process calculations in the oil and gas industry.

Key features used in this workbook:
- **Thermodynamic Models**: Calculation of phase equilibrium, fluid properties (density, viscosity, enthalpy, entropy), and component behavior.
- **Unit Operations**: Simulation of various process equipment such as streams, mixers, heaters, coolers, separators (two-phase and three-phase), compressors, valves, and splitters.
- **Process System**: Construction and simulation of complex process flowsheets by connecting multiple unit operations.

For more information, visit the [NeqSim website](https://neqsim.github.io/).

In [ ]:
facility_name = "demo_field"
simulation_year = 2025

In [ ]:
from dataclasses import dataclass, field
from typing import List
import math
import numpy as np

@dataclass
class ProcessInput:
    # Core
    #  parameters
    Year: int = 2030  # Current year of the simulation
    modus: str = 'normal' # or 'kvasi'  # 'kvasi' or 'normal' (normal includes mixed mode operation)
    design_case: str = 'low'  # Design case 'low', 'intermediate'

    second_fraction: float = 0.0  # Fraction extra flow to seccond stage

    test_sep_pressure: float = 30.0  # bara
    test_sep_temperature: float = 50.0  # C
    test_sep_hc_flow_rate_demo_field: float = 10  # Hydrocarbon flow rate for DEMO_FIELD in kg/day
    test_sep_water_flow_rate_demo_field: float = 1  # Water flow rate for DEMO_FIELD in kg/day
    test_hc_composition_demo_field: List[float] = field(default_factory=lambda: [  # Hydrocarbon composition for DEMO_FIELD
        0.2427, 1.4284, 78.9641, 8.9389, 4.7052, 0.6935, 1.4287, 0.2067,
        0.2469, 0.3546, 0.5473, 0.6345, 0.1843, 0.7085, 0.5630, 0.1528, 0.0
    ])

    first_stage_pressure: float = 30.0
    first_stage_temperature: float = 50.0  # bar
    water_in_oil_1st_stage: float = 0.2  # how much of feed water folows oil from 1st stage separator

    dp_2nd_stage_gas_control_valve: float = 3.5  # bar
    dp_3rd_stage_gas_control_valve: float = 1.0  # bar

    second_stage_pressure: float = 15.0  # bara
    water_in_oil_2nd_stage: float = 0.1  # how much of feed water folows oil from 2nd stage separator

    third_stage_pressure: float = 2.0  # bara

    first_stage_cooler_temperature: float = 25.0  # C
    second_stage_cooler_temperature: float = 25.0  # C
    third_stage_cooler_temperature: float = 25.0  # C

    gas_from_degasing_vessel: float = 500.0  # kg/s

    temperature_1st_stage_injection_cooler: float = 24.0  # C
    pressure_1st_stage_injection_compressor: float = 70.0  # bara
    temperature_2nd_stage_injection_cooler: float = 24.0
    pressure_2nd_stage_injection_compressor: float = 150.0  # bara
    pressuredrop_in_1ststage_inj: float = 0.7 # bara cooler + valve + scrubber
    pressuredrop_in_2ndstage_inj: float = 0.5 # bara cooler + scrubber

    temperature_1st_stage_injection_cooler_B: float = 24.0  # C
    pressure_1st_stage_injection_compressor_B: float = 70.0  # bara
    temperature_2nd_stage_injection_cooler_B: float = 24.0
    pressure_2nd_stage_injection_compressor_B: float = 150.0  # bara
    pressuredrop_in_1ststage_inj_B: float = 0.7 # bara cooler + valve + scrubber
    pressuredrop_in_2ndstage_inj_B: float = 0.5 # bara cooler + scrubber

    temperature_1st_stage_injection_cooler_C: float = 24.0  # C
    pressure_1st_stage_injection_compressor_C: float = 70.0  # bara
    temperature_2nd_stage_injection_cooler_C: float = 24.0
    pressure_2nd_stage_injection_compressor_C: float = 150.0  # bara
    pressuredrop_in_1ststage_inj_C: float = 0.7 # bara cooler + valve + scrubber (Giu: need to update excel)
    pressuredrop_in_2ndstage_inj_C: float = 0.5 # bara cooler + scrubber (Giu: need to update excel)

    gas_injection_manifold_split_factors = [0.25, 0.25, 0.5]

    gas_1st_stage_2nd_stage_bypass: float = 3000.0  # fraction of gas from 1st stage to 2nd stage
    gas_2nd_stage_3rd_stage_bypass: float = 1.0  # fraction of gas from 2nd stage to 3rd stage

    throttle_3rd_stage_recomp: float = 0.0  # bar pressure drop in valve after 3rd stage recompressor
    pressuredrop_in_1ststage: int = 0.4 #pressure drop in 1st stage scrubber
    pressuredrop_in_2ndstage:int = 0.4 #pressure drop in 2nd stage recmpressor cooler and scrubber
    pressuredrop_in_3rdstage: int = 0.4 # pressure drop in 3rd stage recompressor cooler and scrubber

    intermediate_compressor_pressure: float = 15.0  # bara

    closedDrainPressure: float = 1.5  # bara

    hc_flow_rate_demo_field: float = 2.931e6  # Hydrocarbon flow rate for DEMO_FIELD in kg/day
    hc_composition_demo_field: List[float] = field(default_factory=lambda: [  # Hydrocarbon composition for DEMO_FIELD
        0.2427, 1.4284, 78.9641, 8.9389, 4.7052, 0.6935, 1.4287, 0.2067,
        0.2469, 0.3546, 0.5473, 0.6345, 0.1843, 0.7085, 0.5630, 0.1528, 0.0
    ])
    water_flow_rate_demo_field: float = 1.0e5  # Water flow rate for DEMO_FIELD in kg/day

    chartConditions = [] #used to set molecular weight etc.
    headunit = 'kJ/kg'

    speedred: float = 1.0 # fraction of design speed
    base_3rd_stage_impellers: int= 7
    new_3rd_stage_impellers: int = 7

    curve_flow_1st_stage = [4373, 5009, 5407, 5670, 5838, 6075]
    curve_head_1st_stage = [118.34, 116.83, 113.94, 109.89, 104.28, 87.74]

    curve_flow_1st_stage = list(np.array(curve_flow_1st_stage) * speedred * 0.85)
    curve_head_1st_stage = list(np.array(curve_head_1st_stage) * speedred * speedred )

    curve_flow_eff_1st_stage = [4571.05, 5204.23, 5605.83, 5806.77, 6070.03, 6070.01]
    curve_eff_1st_stage = [74.09, 76.17, 76.8, 75.58, 68.98, 68.97]

    curve_flow_2nd_stage = [4633, 4767, 5000.0, 6207, 6598, 7037]
    curve_head_2nd_stage = [133.0, 132.5, 131.5, 124, 114, 89.97]

    curve_flow_2nd_stage = list(np.array(curve_flow_2nd_stage) * speedred )
    curve_head_2nd_stage = list(np.array(curve_head_2nd_stage) * speedred * speedred )

    curve_flow_eff_2nd_stage = [4600.0, 4633.0, 4762.93, 5611.0, 6209.62, 6408.91, 7045.93]
    curve_eff_2nd_stage = [71.0, 71.5, 73.79, 75.34, 74.91, 64.31]

    curve_head_3rd_stage = [146.9, 140.92, 133.66, 118.67]
    curve_flow_3rd_stage = [1179.94, 1498.61, 1699.24, 1953.85]

    curve_flow_eff_3rd_stage = [1170.7, 1501.18, 1699.65, 1948.92]
    curve_eff_3rd_stage = [61.93, 66.27, 67.48, 67.16]

    curve_flow_3rd_stage = list(np.array(curve_flow_3rd_stage) * speedred )
    curve_head_3rd_stage = list(np.array(curve_head_3rd_stage) * speedred * speedred * new_3rd_stage_impellers/base_3rd_stage_impellers )

    #curve_flow_2nd_stage = list(np.array(curve_flow_2nd_stage) * speedred )
    #curve_head_2nd_stage = list(np.array(curve_head_2nd_stage) * speedred * speedred * new_3rd_stage_impellers/base_3rd_stage_impellers )

    curve_flow_1st_stage_inj_A = [2049, 1937, 1788, 1639, 1489, 1341, 1192, 1165]  # single speed
    curve_head_1st_stage_inj_A = [82.2, 90.6, 96.3, 101.2, 105.5, 109, 111.8, 112.2]

    curve_flow_2nd_stage_inj_A = [742, 713, 658, 603, 548, 547, 493, 439, 435]  # single speed
    curve_head_2nd_stage_inj_A = [44.2, 47.6, 52.2, 55.7, 58.5, 58.5, 60.8, 62.7, 62.8]

    curve_flow_1st_stage_inj_B = [1154, 1180, 1328, 1474, 1623, 1770, 1918, 2029]  # single speed
    curve_head_1st_stage_inj_B = [110, 109.6, 106.9, 103.4, 99.2, 94.4, 88.8, 80.6]

    curve_flow_2nd_stage_inj_B = [444, 448, 504, 559, 560, 616, 672, 728, 758]  # single speed
    curve_head_2nd_stage_inj_B = [65.5, 65.4, 63.4, 61.1, 61, 58.1, 54.5, 49.7, 46.1]

    curve_flow_1st_stage_inj_C = [2467, 2499, 2811, 3123, 3436, 3748, 3945]  # single speed
    curve_head_1st_stage_inj_C = [120.4, 120.1, 116.3, 111.5, 105.6, 98.6, 93.3]

    curve_flow_2nd_stage_inj_C = [1000, 1026, 1154, 1282, 1410, 1539, 1667, 1795, 1825]  # single speed
    curve_head_2nd_stage_inj_C = [90.7, 90.3, 87.8, 84.7, 81, 76.7, 71.5, 64.7, 62.4]

inp = ProcessInput()
inp.Year = simulation_year

# flow flow * speed/speed_design
# head = head * (speed/speed_design)^2
# power = power * (speed/speed_design)^3

# Read process data

In [ ]:
import pandas as pd

def read_process_data_from_excel(filename='process_conditions.xlsx', year=None):
    """
    Read process data from Excel file for a given year.

    Parameters:
    filename (str): Name of the Excel file
    year (int): Year to read data for. If None, uses the year from inp.Year

    Returns:
    ProcessInput: Updated ProcessInput object with data from Excel
    """
    try:
        # Read the Excel file
        df = pd.read_excel(filename)

        # Use the year from inp if not provided
        if year is None:
            year = inp.Year

        # Filter data for the specified year
        year_data = df[df['Year'] == year]

        if year_data.empty:
            print(f"Warning: No data found for year {year}. Available years: {sorted(df['Year'].unique())}")
            return inp

        # Get the first (and should be only) row for this year
        row = year_data.iloc[0]

        # Create a new ProcessInput object with updated values
        updated_inp = ProcessInput()

        # Update all matching fields from Excel data
        for column in df.columns:
            if column != 'Year' and hasattr(updated_inp, column):
                setattr(updated_inp, column, row[column])

        # Set the year
        updated_inp.Year = year

        print(f"Successfully loaded data for year {year}")
        print(f"Updated fields: {[col for col in df.columns if col != 'Year' and hasattr(updated_inp, column)]}")

        return updated_inp

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return inp
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return inp

def read_well_stream_data_from_excel(filename, year=None):
    """
    Read well stream composition and flow rate data from Excel file for a given year.

    Parameters:
    filename (str): Name of the Excel file containing well stream data
    year (int): Year to read data for. If None, uses the year from inp.Year

    Returns:
    tuple: (hc_composition_demo_field, hc_flow_rate_demo_field, water_flow_rate_demo_field)
    """
    try:
        # Read the Excel file with the first row as header
        df = pd.read_excel(filename, header=0)

        # Remove the units row (first row after header)
        df = df[df['date '] != 'year']

        # Convert date column to numeric
        df['date '] = pd.to_numeric(df['date '], errors='coerce')

        # Use the year from inp if not provided
        if year is None:
            year = inp.Year

        # Filter data for the specified year
        year_data = df[df['date '] == year]

        if year_data.empty:
            print(f"Warning: No well stream data found for year {year}. Available years: {sorted(df['date '].dropna().unique())}")
            # Return current values as fallback
            return inp.hc_composition_demo_field, inp.hc_flow_rate_demo_field, inp.water_flow_rate_demo_field

        # Get the first (and should be only) row for this year
        row = year_data.iloc[0]

        # Define the composition columns in the correct order (matching NeqSim order)
        # Note: we include n2 and exclude it will be set to 0, but we need to map correctly
        composition_columns = ['n2', 'CO2 ', 'c1', 'c2', 'c3 ', 'ic4 ', 'c4', 'ic5 ', 'c5', 'C6 ', 'F1 ', 'F2', 'F3', 'F4', 'F5', 'F6']

        # Read composition data (convert from mol% to mole fractions)
        hc_composition = []
        for col in composition_columns:
            if col in df.columns:
                # Convert from mol% to mole fraction
                value = row[col]
                if pd.isna(value):
                    hc_composition.append(0.0)
                else:
                    hc_composition.append(float(value) / 100.0)
            else:
                print(f"Warning: Column '{col}' not found in well stream data")
                hc_composition.append(0.0)

        # Add water component (assuming it's the last component, set to 0 for hydrocarbon stream)
        hc_composition.append(0.0)

        # Read flow rates from the Excel file
        # Convert from the units in the file to kg/day
        oil_rate_sm3_per_day = row.get('oil rate ', 0.0)  # sm3/day
        gas_rate_sm3_per_day = row.get('gas rate ', 0.0)  # sm3/day
        mass_rate_kg_per_day = row.get('mass rate ', inp.hc_flow_rate_demo_field)  # kg/day
        mass_rate_kg_per_day = float(mass_rate_kg_per_day) if not pd.isna(mass_rate_kg_per_day) else inp.hc_flow_rate_demo_field

        # Use mass rate if available, otherwise calculate from oil and gas rates
        if not pd.isna(mass_rate_kg_per_day) and mass_rate_kg_per_day > 0:
            hc_flow_rate = float(mass_rate_kg_per_day)
        else:
            # If mass rate is not available, use the default
            hc_flow_rate = inp.hc_flow_rate_demo_field

        # For water flow rate, we'll use the default for now unless specified in the file
        water_flow_rate = inp.water_flow_rate_demo_field

        print(f"Successfully loaded well stream data for year {year}")
        print(f"HC flow rate: {hc_flow_rate} kg/day")
        print(f"Water flow rate: {water_flow_rate} kg/day")
        print(f"HC composition (mol fractions): {[f'{comp:.4f}' for comp in hc_composition[:5]]}... (showing first 5)")

        # Normalize composition to ensure it sums to 1.0
        total = sum(hc_composition)
        if total > 0:
            hc_composition = [comp/total for comp in hc_composition]
            print(f"Composition normalized to sum = 1.0 (was {total:.4f})")

        return hc_composition, hc_flow_rate, water_flow_rate

    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        return inp.hc_composition_demo_field, inp.hc_flow_rate_demo_field, inp.water_flow_rate_demo_field
    except Exception as e:
        print(f"Error reading well stream Excel file: {e}")
        return inp.hc_composition_demo_field, inp.hc_flow_rate_demo_field, inp.water_flow_rate_demo_field

def show_available_years(filename='process_conditions.xlsx'):
    """Show all available years in the Excel file."""
    try:
        df = pd.read_excel(filename)
        years = sorted(df['Year'].unique())
        print(f"Available years in {filename}: {years}")
        return years
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

def show_available_well_stream_years(filename='well_stream/well_stream_rates.xlsx'):
    """Show all available years in the well stream Excel file."""
    try:
        df = pd.read_excel(filename, header=0)
        # Remove the units row
        df = df[df['date '] != 'year']
        # Convert to numeric
        df['date '] = pd.to_numeric(df['date '], errors='coerce')

        years = sorted(df['date '].dropna().unique())
        print(f"Available years in {filename}: {[int(y) for y in years]}")
        return [int(y) for y in years]
    except Exception as e:
        print(f"Error reading file: {e}")
        return []

def update_inp_for_year(target_year):
    """Convenience function to update inp for a specific year."""
    global inp
    # Read process conditions
    inp = read_process_data_from_excel(year=target_year)

    # Read well stream data and update inp
    hc_composition, hc_flow_rate, water_flow_rate = read_well_stream_data_from_excel(filename='well_stream/well_stream_rates.xlsx', year=target_year)
    inp.hc_composition_demo_field = hc_composition
    inp.hc_flow_rate_demo_field = hc_flow_rate
    inp.water_flow_rate_demo_field = water_flow_rate

    return inp

# Show available years
print("Available data:")
available_years = show_available_years()
print("\nAvailable well stream data:")
available_well_years = show_available_well_stream_years()

# Read data for the specified year and update inp
inp = read_process_data_from_excel(year=inp.Year)

# Read well stream data and update inp
hc_composition, hc_flow_rate, water_flow_rate = read_well_stream_data_from_excel(filename='well_stream/well_stream_rates.xlsx', year=inp.Year)
inp.hc_composition_demo_field = hc_composition
inp.hc_flow_rate_demo_field = hc_flow_rate
inp.water_flow_rate_demo_field = water_flow_rate

if(inp.design_case == 'intermediate'):
    # Read well stream data and update inp
    hc_composition, hc_flow_rate, water_flow_rate = read_well_stream_data_from_excel(filename='well_stream/interm_well_stream_rates.xlsx', year=inp.Year)
    inp.hc_composition_demo_field = hc_composition
    inp.hc_flow_rate_demo_field = hc_flow_rate
    inp.water_flow_rate_demo_field = water_flow_rate

#If use intermediate case
test_sep_hc_composition, test_sep_hc_flow_rate, test_sep_water_flow_rate = read_well_stream_data_from_excel(filename='well_stream/well_stream_rates_test_sep.xlsx', year=inp.Year)
inp.test_sep_hc_composition_demo_field = test_sep_hc_composition
inp.test_sep_hc_flow_rate_demo_field = test_sep_hc_flow_rate
inp.water_flow_rate_demo_field = test_sep_water_flow_rate

print(f"\nCurrent process parameters for year {inp.Year}:")
print(f"First stage pressure: {inp.first_stage_pressure} bara")
print(f"First stage temperature: {inp.first_stage_temperature} °C")
print(f"Second stage pressure: {inp.second_stage_pressure} bara")
print(f"Water in oil 1st stage: {inp.water_in_oil_1st_stage} vol frac")
print(f"HC flow rate: {inp.hc_flow_rate_demo_field} kg/day")
print(f"Water flow rate: {inp.water_flow_rate_demo_field} kg/day")

print(f"\nCurrent process parameters for year {inp.Year}:")
print(f"Test separator pressure: {inp.test_sep_pressure} bara")
print(f"Test separator temperature: {inp.test_sep_temperature} °C")
print(f"Test separator water in oil 1st stage: {inp.water_in_oil_1st_stage} vol frac")
print(f"Test separator HC flow rate: {inp.test_sep_hc_flow_rate_demo_field} kg/day")
print(f"Test separator water flow rate: {inp.test_sep_water_flow_rate_demo_field} kg/day")

In [ ]:
# Display the complete composition data for verification
print(f"Complete HC composition for year {inp.Year}:")
composition_names = [
    'N2', 'CO2', 'C1', 'C2', 'C3', 'iC4', 'C4', 'iC5', 'C5', 'C6', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'H2O']

print("\nComponent\tMol Fraction\tMol %")
print("-" * 40)
for i, (name, frac) in enumerate(zip(composition_names, inp.hc_composition_demo_field)):
    print(f"{name}\t\t{frac:.6f}\t{frac*100:.4f}")

print(f"\nTotal: {sum(inp.hc_composition_demo_field):.6f}")

# Import fluid

In [ ]:
from neqsim.process.processTools import stream, mixer
from neqsim import jneqsim as neqsim

# Load base well fluid
def create_fluid(composition=None):
    wellFluid = neqsim.thermo.util.readwrite.EclipseFluidReadWrite.read('DemoAsset/data/demo_field_fluid.e300')
    if composition is not None:
        wellFluid.setMolarComposition(composition)
    wellFluid.setMultiPhaseCheck(True)
    wellFluid.init(0)
    return wellFluid

def create_water_fluid():
    """Create a pure water fluid based on the base fluid."""
    water_fluid = create_fluid()
    n = water_fluid.getNumberOfComponents()
    composition = [0] * n
    composition[-1] = 1  # Assume last component is water
    water_fluid.setMolarComposition(composition)
    water_fluid.init(0)
    return water_fluid

def create_gas_recovery_fluid():
    """Create a pure gas recovery fluid based on the base fluid."""
    gas_recovery_fluid = create_fluid()
    n = gas_recovery_fluid.getNumberOfComponents()
    composition = [0] * n
    composition[0] = 0.01 #N2
    composition[1] = 0.06 #CO2
    composition[2] = 0.82  #CH4
    composition[3] = 0.06
    composition[4] = 0.03  #C3
    gas_recovery_fluid.setMolarComposition(composition)
    gas_recovery_fluid.init(0)
    return gas_recovery_fluid

def create_stream(name, composition=None):
    """Create a stream with a given fluid and optional composition."""
    fluid = create_fluid(composition)
    return neqsim.process.equipment.stream.Stream(name, fluid)

def create_water_stream(name, composition=None):
    """Create a stream with a given fluid and optional composition."""
    fluid = create_water_fluid()
    return neqsim.process.equipment.stream.Stream(name, fluid)

In [ ]:
from neqsim.thermo import printFrame, TPflash
demo_field_fluid = create_fluid()
water_fluid = create_water_fluid()
# printFrame(demo_field_fluid)
#TPflash(water_fluid)
#printFrame(water_fluid)

# Create well stream

In [ ]:
def createFeedProcess(inp):
    demo_field_process = neqsim.process.processmodel.ProcessSystem()

    demo_fieldCTestFeed = create_stream('Demo Field C Test sep Feed')
    demo_fieldCTestFeed.setTemperature(inp.test_sep_temperature, 'C')
    demo_fieldCTestFeed.setPressure(inp.test_sep_pressure, 'bara')
    demo_fieldCTestFeed.getFluid().setMolarComposition(inp.test_hc_composition_demo_field) # if two different composition change this
    demo_fieldCTestFeed.setFlowRate(inp.test_sep_hc_flow_rate_demo_field, 'kg/day')
    demo_fieldCTestFeed.run()
    demo_field_process.add(demo_fieldCTestFeed)

    demo_fieldCWaterTestFeed = create_water_stream('Demo Field C Test sep Water Feed')
    demo_fieldCWaterTestFeed.setTemperature(inp.test_sep_temperature, 'C')
    demo_fieldCWaterTestFeed.setPressure(inp.test_sep_pressure, 'bara')
    demo_fieldCWaterTestFeed.setFlowRate(inp.test_sep_water_flow_rate_demo_field*1000, 'kg/hr')
    demo_fieldCWaterTestFeed.run()
    demo_field_process.add(demo_fieldCWaterTestFeed)

    test_feed_mixer = neqsim.process.equipment.mixer.Mixer('test feed mixer')
    test_feed_mixer.addStream(demo_fieldCTestFeed)
    test_feed_mixer.addStream(demo_fieldCWaterTestFeed)
    test_feed_mixer.run()
    demo_field_process.add(test_feed_mixer)

    test_feed_heater = neqsim.process.equipment.heatexchanger.Heater(
        'test feed heater')
    test_feed_heater.setInletStream(test_feed_mixer.getOutStream())
    test_feed_heater.setOutTemperature(inp.test_sep_temperature, 'C')
    test_feed_heater.run()
    demo_field_process.add(test_feed_heater)

    testSeparator = neqsim.process.equipment.separator.ThreePhaseSeparator(
        "Test separator", test_feed_heater.getOutStream())
    testSeparator.setEntrainment(
        inp.water_in_oil_1st_stage, "feed", "volume", "aqueous", "oil")
    try:
        testSeparator.run()
    except:
        print('error in Test separator')
    demo_field_process.add(testSeparator)

    water_out_test_separator = neqsim.process.equipment.stream.Stream('water out test separator', testSeparator.getWaterOutStream())
    water_out_test_separator.run()
    demo_field_process.add(water_out_test_separator)

    test_gas_splitter = neqsim.process.equipment.splitter.Splitter("test gas splitter")
    test_gas_splitter.setInletStream(testSeparator.getGasOutStream())

    if inp.modus == 'kvasi':
       test_gas_splitter.setSplitFactors([0, 1.0])
    else:
       test_gas_splitter.setSplitFactors([1.0, 0.0])

    test_gas_splitter.run()
    demo_field_process.add(test_gas_splitter)

    demo_fieldCFeed_2nd = create_stream('Demo Field C Feed 2nd')
    demo_fieldCFeed_2nd.setTemperature(inp.first_stage_temperature, 'C')
    demo_fieldCFeed_2nd.setPressure(inp.second_stage_pressure, 'bara')
    demo_fieldCFeed_2nd.getFluid().setMolarComposition(inp.hc_composition_demo_field)
    demo_fieldCFeed_2nd.setFlowRate(inp.hc_flow_rate_demo_field*inp.second_fraction, 'kg/day')
    demo_fieldCFeed_2nd.run()
    demo_field_process.add(demo_fieldCFeed_2nd)

    demo_fieldCWaterFeed_2nd = create_water_stream('Demo Field C Water Feed 2nd')
    demo_fieldCWaterFeed_2nd.setTemperature(inp.first_stage_temperature, 'C')
    demo_fieldCWaterFeed_2nd.setPressure(inp.second_stage_pressure, 'bara')
    demo_fieldCWaterFeed_2nd.setFlowRate(inp.water_flow_rate_demo_field*1000*inp.second_fraction, 'kg/day')
    demo_fieldCWaterFeed_2nd.run()
    demo_field_process.add(demo_fieldCWaterFeed_2nd)

    feed_mixer_2nd = neqsim.process.equipment.mixer.Mixer('feed mixer 2nd')
    feed_mixer_2nd.addStream(demo_fieldCFeed_2nd)
    feed_mixer_2nd.addStream(demo_fieldCWaterFeed_2nd)
    feed_mixer_2nd.run()
    demo_field_process.add(feed_mixer_2nd)

    feed_heater_2nd = neqsim.process.equipment.heatexchanger.Heater(
        'feed heater 2nd')
    feed_heater_2nd.setInletStream(feed_mixer_2nd.getOutStream())
    feed_heater_2nd.setOutTemperature(inp.first_stage_temperature, 'C')
    feed_heater_2nd.run()
    demo_field_process.add(feed_heater_2nd)

    demo_fieldCFeed = create_stream('Demo Field C Feed')
    demo_fieldCFeed.setTemperature(inp.first_stage_temperature, 'C')
    demo_fieldCFeed.setPressure(inp.first_stage_pressure, 'bara')
    demo_fieldCFeed.getFluid().setMolarComposition(inp.hc_composition_demo_field)
    demo_fieldCFeed.setFlowRate(inp.hc_flow_rate_demo_field, 'kg/day')
    demo_fieldCFeed.run()
    demo_field_process.add(demo_fieldCFeed)

    demo_fieldCWaterFeed = create_water_stream('Demo Field C Water Feed')
    demo_fieldCWaterFeed.setTemperature(inp.first_stage_temperature, 'C')
    demo_fieldCWaterFeed.setPressure(inp.first_stage_pressure, 'bara')
    demo_fieldCWaterFeed.setFlowRate(inp.water_flow_rate_demo_field*1000, 'kg/day')
    demo_fieldCWaterFeed.run()
    demo_field_process.add(demo_fieldCWaterFeed)

    feed_mixer = neqsim.process.equipment.mixer.Mixer('feed mixer')
    feed_mixer.addStream(demo_fieldCFeed)
    feed_mixer.addStream(demo_fieldCWaterFeed)
    feed_mixer.run()
    demo_field_process.add(feed_mixer)

    feed_heater = neqsim.process.equipment.heatexchanger.Heater(
        'feed heater')
    feed_heater.setInletStream(feed_mixer.getOutStream())
    feed_heater.setOutTemperature(inp.first_stage_temperature, 'C')
    feed_heater.run()
    demo_field_process.add(feed_heater)

    firstStageSeparator = neqsim.process.equipment.separator.ThreePhaseSeparator(
        "1st stage separator", feed_heater.getOutStream())
    firstStageSeparator.setEntrainment(
        inp.water_in_oil_1st_stage, "feed", "volume", "aqueous", "oil")
    try:
        firstStageSeparator.run()
    except:
        print('error in 1st stage separator')
    demo_field_process.add(firstStageSeparator)

    gas_splitter = neqsim.process.equipment.splitter.Splitter(
        'gas splitter')
    gas_splitter.setInletStream(firstStageSeparator.getGasOutStream())
    gas_splitter.setFlowRates([-1, inp.gas_1st_stage_2nd_stage_bypass], "kg/hr")
    gas_splitter.run()
    demo_field_process.add(gas_splitter)

    water_out_1st_stage = neqsim.process.equipment.stream.Stream('water out 1st stage', firstStageSeparator.getWaterOutStream())
    water_out_1st_stage.run()
    demo_field_process.add(water_out_1st_stage)

    oilHeaterFromFirstStage = neqsim.process.equipment.heatexchanger.Heater(
        "oil heater second stage", firstStageSeparator.getOilOutStream())
    oilHeaterFromFirstStage.setdT(0.0)
    oilHeaterFromFirstStage.run()
    demo_field_process.add(oilHeaterFromFirstStage)

    valve_oil_from_first_stage = neqsim.process.equipment.valve.ThrottlingValve(
        "valve oil from first stage", oilHeaterFromFirstStage.getOutStream())
    valve_oil_from_first_stage.setOutletPressure(
        inp.second_stage_pressure, 'bar')
    valve_oil_from_first_stage.run()
    demo_field_process.add(valve_oil_from_first_stage)

    recycleliqstream2  = create_stream('recycle 2nd stage')
    recycleliqstream2.setPressure(inp.second_stage_pressure, 'bara')
    recycleliqstream2.setTemperature(25.0, 'C')
    recycleliqstream2.setFlowRate(1.0, "kg/hr")
    recycleliqstream2.run()
    demo_field_process.add(recycleliqstream2)

    recycleliqstream2A  = create_stream('recycle 2nd stage A')
    recycleliqstream2A.setPressure(inp.second_stage_pressure, 'bara')
    recycleliqstream2A.setTemperature(25.0, 'C')
    recycleliqstream2A.setFlowRate(1.0, "kg/hr")
    recycleliqstream2A.run()
    demo_field_process.add(recycleliqstream2A)

    recycleliqstream2B  = create_stream('recycle 2nd stage B')
    recycleliqstream2B.setPressure(inp.second_stage_pressure, 'bara')
    recycleliqstream2B.setTemperature(25.0, 'C')
    recycleliqstream2B.setFlowRate(1.0, "kg/hr")
    recycleliqstream2B.run()
    demo_field_process.add(recycleliqstream2B)

    recycleliqstream2C  = create_stream('recycle 2nd stage C')
    recycleliqstream2C.setPressure(inp.second_stage_pressure, 'bara')
    recycleliqstream2C.setTemperature(25.0, 'C')
    recycleliqstream2C.setFlowRate(1.0, "kg/hr")
    recycleliqstream2C.run()
    demo_field_process.add(recycleliqstream2C)

    splittergasto2ndstage   = neqsim.process.equipment.splitter.Splitter('splitter gas to 2nd stage',gas_splitter.getSplitStream(1))
    splittergasto2ndstage.setSplitFactors([0.999, 0.001])
    splittergasto2ndstage.run()
    demo_field_process.add(splittergasto2ndstage)

    secondStageSeparator = neqsim.process.equipment.separator.ThreePhaseSeparator(
        "2nd stage separator", valve_oil_from_first_stage.getOutStream())
    secondStageSeparator.addStream(testSeparator.getOilOutStream())
    secondStageSeparator.addStream(splittergasto2ndstage.getSplitStream(0))
    secondStageSeparator.addStream(feed_heater_2nd.getOutStream())
    secondStageSeparator.addStream(recycleliqstream2)
    secondStageSeparator.addStream(recycleliqstream2A)
    secondStageSeparator.addStream(recycleliqstream2B)
    secondStageSeparator.addStream(recycleliqstream2C)
    secondStageSeparator.setEntrainment(
        inp.water_in_oil_2nd_stage, "feed", "volume", "aqueous", "oil")
    secondStageSeparator.run()
    demo_field_process.add(secondStageSeparator)


    secondStageSeparatorGasMixer = neqsim.process.equipment.mixer.Mixer('2nd stage separator gas mixer')
    secondStageSeparatorGasMixer.addStream(secondStageSeparator.getGasOutStream())
    secondStageSeparatorGasMixer.addStream(splittergasto2ndstage.getSplitStream(1))
    secondStageSeparatorGasMixer.run()
    demo_field_process.add(secondStageSeparatorGasMixer)

    seccondstagegassplitter = neqsim.process.equipment.splitter.Splitter(
        'second stage gas splitter')
    seccondstagegassplitter.setInletStream(secondStageSeparatorGasMixer.getOutStream())
    seccondstagegassplitter.setFlowRates([-1, inp.gas_2nd_stage_3rd_stage_bypass], "kg/hr")
    seccondstagegassplitter.run()
    demo_field_process.add(seccondstagegassplitter)

    water_out_2nd_stage = neqsim.process.equipment.stream.Stream('water out 2nd stage', secondStageSeparator.getWaterOutStream())
    water_out_2nd_stage.run()
    demo_field_process.add(water_out_2nd_stage)

    valve_oil_from_second_stage = neqsim.process.equipment.valve.ThrottlingValve(
        "valve oil from second stage", secondStageSeparator.getOilOutStream())
    valve_oil_from_second_stage.setOutletPressure(
        inp.third_stage_pressure, 'bara')
    valve_oil_from_second_stage.run()
    demo_field_process.add(valve_oil_from_second_stage)

    recycleliqstream3  = create_stream('recycle 3rd stage')
    recycleliqstream3.setPressure(inp.third_stage_pressure, 'bara')
    recycleliqstream3.setTemperature(25.0, 'C')
    recycleliqstream3.setFlowRate(1.0, "kg/hr")
    recycleliqstream3.run()
    demo_field_process.add(recycleliqstream3)

    thirdStageSeparator = neqsim.process.equipment.separator.ThreePhaseSeparator(
        "3RD stage separator", valve_oil_from_second_stage.getOutStream())
    thirdStageSeparator.addStream(recycleliqstream3)
    thirdStageSeparator.run()
    demo_field_process.add(thirdStageSeparator)

    exportoil_stream = neqsim.process.equipment.stream.Stream('export oil', thirdStageSeparator.getOilOutStream())
    exportoil_stream.run()
    demo_field_process.add(exportoil_stream)

    firstStagePressureControllValve = neqsim.process.equipment.valve.ThrottlingValve('1st stage pressure control valve', thirdStageSeparator.getGasOutStream())
    firstStagePressureControllValve.setOutletPressure(inp.third_stage_pressure-inp.dp_3rd_stage_gas_control_valve, "bara")
    firstStagePressureControllValve.run()
    demo_field_process.add(firstStagePressureControllValve)

    recyclegasstream1  = create_stream('recycle gas 1st stage')
    recyclegasstream1.setPressure(inp.third_stage_pressure-inp.pressuredrop_in_1ststage, 'bara')
    recyclegasstream1.setTemperature(25.0, 'C')
    recyclegasstream1.setFlowRate(10.0, "kg/hr")
    recyclegasstream1.run()
    demo_field_process.add(recyclegasstream1)

    gas_recovery_fluid = create_gas_recovery_fluid()
    gas_recovery_stream = neqsim.process.equipment.stream.Stream('gas recovery stream', gas_recovery_fluid)
    gas_recovery_stream.setPressure(inp.third_stage_pressure, 'bara')
    gas_recovery_stream.setTemperature(25.0, 'C')
    gas_recovery_stream.setFlowRate(inp.gas_from_degasing_vessel, "kg/hr")
    gas_recovery_stream.run()
    demo_field_process.add(gas_recovery_stream)

    gas_mixer = neqsim.process.equipment.mixer.Mixer('gas mixer 0')
    gas_mixer.addStream(firstStagePressureControllValve.getOutStream())
    gas_mixer.addStream(recyclegasstream1)
    gas_mixer.addStream(gas_recovery_stream)
    gas_mixer.addStream(seccondstagegassplitter.getSplitStream(1))
    gas_mixer.run()
    demo_field_process.add(gas_mixer)

    firstStageScrubber0 = neqsim.process.equipment.separator.GasScrubber(
        "VG-23-002", gas_mixer.getOutStream())
    #firstStageScrubber0.setEntrainment(
    #    0.1, "feed", "volume", "oil", "gas")
    firstStageScrubber0.setInternalDiameter(0.762)
    firstStageScrubber0.run()
    demo_field_process.add(firstStageScrubber0)

    liqControlValveToDrain = neqsim.process.equipment.valve.ThrottlingValve('liquid control valve to drain', firstStageScrubber0.getLiquidOutStream())
    liqControlValveToDrain.setOutletPressure(inp.closedDrainPressure, "bara")
    liqControlValveToDrain.setPercentValveOpening(50.0)
    liqControlValveToDrain.setTagName('23-FV-002')
    liqControlValveToDrain.run()
    demo_field_process.add(liqControlValveToDrain)

    stream_to_closeddrain = neqsim.process.equipment.stream.Stream('stream to closed drain', liqControlValveToDrain.getOutStream())
    stream_to_closeddrain.run()
    demo_field_process.add(stream_to_closeddrain)

    firstStageCompressor = neqsim.process.equipment.compressor.Compressor(
        "23-KA-004", firstStageScrubber0.getGasOutStream())
    firstStageCompressor.setCompressorChartType("interpolate and extrapolate")
    firstStageCompressor.setUsePolytropicCalc(True)
    firstStageCompressor.setPolytropicEfficiency(0.7) #based on performance test 2024 (changed from 0.6 11.08.2025)
    firstStageCompressor.setOutletPressure(inp.second_stage_pressure-inp.dp_2nd_stage_gas_control_valve, 'bara')
    firstStageCompressor.setSpeed(10000)
    firstStageCompressor.getCompressorChart().setCurves(inp.chartConditions, [10000],[inp.curve_flow_1st_stage], [inp.curve_head_1st_stage], [inp.curve_flow_eff_1st_stage], [inp.curve_eff_1st_stage])
    firstStageCompressor.getCompressorChart().setUseCompressorChart(False)
    firstStageCompressor.setUseEnergyEfficiencyChart(True)
    firstStageCompressor.getCompressorChart().getSurgeCurve().setCurve(inp.chartConditions, inp.curve_flow_1st_stage, inp.curve_head_1st_stage)
    firstStageCompressor.run()
    demo_field_process.add(firstStageCompressor)

    gassplitter_1st_stage = neqsim.process.equipment.splitter.Splitter(
    '1st stage anti surge splitter')
    gassplitter_1st_stage.setInletStream(firstStageCompressor.getOutletStream())
    gassplitter_1st_stage.setFlowRates([-1, 1.0], "kg/hr")
    gassplitter_1st_stage.run()
    demo_field_process.add(gassplitter_1st_stage)

    antisurgeCalculator = neqsim.process.equipment.util.Calculator("anti surge calculator_1")
    antisurgeCalculator.addInputVariable(firstStageCompressor)
    antisurgeCalculator.setOutputVariable(gassplitter_1st_stage)
    antisurgeCalculator.run()
    demo_field_process.add(antisurgeCalculator)

    antisurgevalve = neqsim.process.equipment.valve.ThrottlingValve('aniti surge valve', gassplitter_1st_stage.getSplitStream(1))
    antisurgevalve.setOutletPressure(inp.third_stage_pressure-inp.pressuredrop_in_1ststage-inp.dp_3rd_stage_gas_control_valve, "bara")
    antisurgevalve.run()
    demo_field_process.add(antisurgevalve)

    anti_surge_cooler = neqsim.process.equipment.heatexchanger.Cooler(
        "anti surge cooler", antisurgevalve.getOutletStream())
    anti_surge_cooler.setOutTemperature(inp.first_stage_cooler_temperature, 'C')
    anti_surge_cooler.run()
    demo_field_process.add(anti_surge_cooler)

    recycl = neqsim.process.equipment.util.Recycle("recycle anti surge 1st stage compressor")
    recycl.addStream(anti_surge_cooler.getOutletStream())
    recycl.setOutletStream(recyclegasstream1)
    recycl.setTolerance(1e-2)
    recycl.run()
    demo_field_process.add(recycl)

    recyclegasstream2  = create_stream('recycle gas 2nd stage')
    recyclegasstream2.setPressure(inp.second_stage_pressure-inp.pressuredrop_in_2ndstage-inp.dp_2nd_stage_gas_control_valve, 'bara')
    recyclegasstream2.setTemperature(25.0, 'C')
    recyclegasstream2.setFlowRate(10.0, "kg/hr")
    recyclegasstream2.run()
    demo_field_process.add(recyclegasstream2)

    gas_mixer2 = neqsim.process.equipment.mixer.Mixer('gas mixer 22')
    gas_mixer2.addStream(gassplitter_1st_stage.getSplitStream(0))
    gas_mixer2.addStream(recyclegasstream2)
    gas_mixer2.addStream(seccondstagegassplitter.getSplitStream(0))
    gas_mixer2.addStream(test_gas_splitter.getSplitStream(1))
    gas_mixer2.run()
    demo_field_process.add(gas_mixer2)

    firstStageCooler = neqsim.process.equipment.heatexchanger.Cooler(
        "HA-23-005", gas_mixer2.getOutStream())
    firstStageCooler.setOutTemperature(
        inp.second_stage_cooler_temperature, 'C')
    firstStageCooler.run()
    demo_field_process.add(firstStageCooler)

    firstStageScrubber1 = neqsim.process.equipment.separator.GasScrubber(
        "VG-23-006", firstStageCooler.getOutStream())
    #firstStageScrubber1.setEntrainment(
    #    0.1, "feed", "volume", "oil", "gas")
    firstStageScrubber1.setOrientation("vertical")
    firstStageScrubber1.setInternalDiameter(1.07)
    firstStageScrubber1.run()
    demo_field_process.add(firstStageScrubber1)

    liqControlValveFirstStage = neqsim.process.equipment.valve.ThrottlingValve('liquid control valve first stage', firstStageScrubber1.getLiquidOutStream())
    liqControlValveFirstStage.setOutletPressure(inp.third_stage_pressure, "bara")
    liqControlValveFirstStage.setPercentValveOpening(50.0)
    liqControlValveFirstStage.setTagName('23-FV-006')
    liqControlValveFirstStage.run()
    demo_field_process.add(liqControlValveFirstStage)

    recycl = neqsim.process.equipment.util.Recycle("recycle 1st stage liquid")
    recycl.addStream(liqControlValveFirstStage.getOutletStream())
    recycl.setOutletStream(recycleliqstream3)
    recycl.setTolerance(1e-2)
    recycl.run()
    demo_field_process.add(recycl)

    secStageCompressor = neqsim.process.equipment.compressor.Compressor(
        "23-KA-007", firstStageScrubber1.getGasOutStream())
    secStageCompressor.setCompressorChartType("interpolate and extrapolate")
    secStageCompressor.setUsePolytropicCalc(True)
    secStageCompressor.setPolytropicEfficiency(0.7) #based on performance test 2024 (changed from 0.6 11.08.2025)
    secStageCompressor.setOutletPressure(inp.intermediate_compressor_pressure, 'bara')
    secStageCompressor.setSpeed(10000) #rpm
    secStageCompressor.getCompressorChart().setCurves(inp.chartConditions, [10000],[inp.curve_flow_2nd_stage], [inp.curve_head_2nd_stage], [inp.curve_flow_eff_2nd_stage], [inp.curve_eff_2nd_stage])
    secStageCompressor.getCompressorChart().setUseCompressorChart(False)
    secStageCompressor.setUseEnergyEfficiencyChart(True)
    secStageCompressor.getCompressorChart().getSurgeCurve().setCurve(inp.chartConditions, inp.curve_flow_2nd_stage, inp.curve_head_2nd_stage)
    secStageCompressor.run()
    demo_field_process.add(secStageCompressor)

    gassplitter_2nd_stage = neqsim.process.equipment.splitter.Splitter(
    '2nd stage anti surge splitter')
    gassplitter_2nd_stage.setInletStream(secStageCompressor.getOutletStream())
    gassplitter_2nd_stage.setFlowRates([-1, 1.0], "kg/hr")
    gassplitter_2nd_stage.run()
    demo_field_process.add(gassplitter_2nd_stage)

    antisurgeCalculator2 = neqsim.process.equipment.util.Calculator("anti surge calculator 2nd stage")
    antisurgeCalculator2.addInputVariable(secStageCompressor)
    antisurgeCalculator2.setOutputVariable(gassplitter_2nd_stage)
    antisurgeCalculator2.run()
    demo_field_process.add(antisurgeCalculator2)

    antisurgevalve2 = neqsim.process.equipment.valve.ThrottlingValve('aniti surge valve 2nd stage', gassplitter_2nd_stage.getSplitStream(1))
    antisurgevalve2.setOutletPressure(inp.second_stage_pressure-inp.pressuredrop_in_2ndstage-inp.dp_2nd_stage_gas_control_valve, "bara")
    antisurgevalve2.run()
    demo_field_process.add(antisurgevalve2)

    recyc_2nd_anti = neqsim.process.equipment.util.Recycle("recycle anti surge 2nd stage compressor")
    recyc_2nd_anti.addStream(antisurgevalve2.getOutletStream())
    recyc_2nd_anti.setOutletStream(recyclegasstream2)
    recyc_2nd_anti.setTolerance(1e-2)
    recyc_2nd_anti.run()
    demo_field_process.add(recyc_2nd_anti)

    recyclegasstream12  = create_stream('recycle gas 2nd stage 2')
    recyclegasstream12.setPressure(inp.intermediate_compressor_pressure-inp.pressuredrop_in_3rdstage, 'bara')
    recyclegasstream12.setTemperature(25.0, 'C')
    recyclegasstream12.setFlowRate(1.0, "kg/hr")
    recyclegasstream12.run()
    demo_field_process.add(recyclegasstream12)

    gas1_mixer = neqsim.process.equipment.mixer.Mixer('gas mixer 1')
    gas1_mixer.addStream(gassplitter_2nd_stage.getSplitStream(0))
    gas1_mixer.addStream(recyclegasstream12)
    gas1_mixer.run()
    demo_field_process.add(gas1_mixer)

    secondStageCooler = neqsim.process.equipment.heatexchanger.Cooler(
        "23-HA-006", gas1_mixer.getOutStream())
    secondStageCooler.setOutTemperature(
        inp.third_stage_cooler_temperature, 'C')
    secondStageCooler.run()
    demo_field_process.add(secondStageCooler)

    secondStageScrubber1 = neqsim.process.equipment.separator.GasScrubber(
        "23-VG-009", secondStageCooler.getOutStream())
    #secondStageScrubber1.setEntrainment(
    #    0.1, "feed", "volume", "oil", "gas")
    secondStageScrubber1.setInternalDiameter(0.915)
    secondStageScrubber1.run()
    demo_field_process.add(secondStageScrubber1)

    liqControlValveSecStage = neqsim.process.equipment.valve.ThrottlingValve('liquid control valve second stage', secondStageScrubber1.getLiquidOutStream())
    liqControlValveSecStage.setOutletPressure(inp.second_stage_pressure, "bara")
    liqControlValveSecStage.setPercentValveOpening(50.0)
    liqControlValveSecStage.setTagName('23-FV-009')
    liqControlValveSecStage.run()
    demo_field_process.add(liqControlValveSecStage)

    recyc2 = neqsim.process.equipment.util.Recycle("recycle 2nd stage liquid")
    recyc2.addStream(liqControlValveSecStage.getOutletStream())
    recyc2.setOutletStream(recycleliqstream2)
    recyc2.setTolerance(1e-3)
    recyc2.run()
    demo_field_process.add(recyc2)

    thirdStageCompressor = neqsim.process.equipment.compressor.Compressor(
        "23-KA-010", secondStageScrubber1.getGasOutStream())
    thirdStageCompressor.setCompressorChartType("interpolate and extrapolate")
    thirdStageCompressor.setUsePolytropicCalc(True)
    thirdStageCompressor.setPolytropicEfficiency(0.7)
    thirdStageCompressor.setOutletPressure((inp.first_stage_pressure+inp.throttle_3rd_stage_recomp), 'bara')
    thirdStageCompressor.setSpeed(10000) #rpm
    thirdStageCompressor.getCompressorChart().setCurves(inp.chartConditions, [10000],[inp.curve_flow_3rd_stage], [inp.curve_head_3rd_stage], [inp.curve_flow_eff_3rd_stage], [inp.curve_eff_3rd_stage])
    thirdStageCompressor.getCompressorChart().setUseCompressorChart(False)
    thirdStageCompressor.setUseEnergyEfficiencyChart(True)
    thirdStageCompressor.getCompressorChart().getSurgeCurve().setCurve(inp.chartConditions, inp.curve_flow_3rd_stage, inp.curve_head_3rd_stage)
    thirdStageCompressor.run()
    demo_field_process.add(thirdStageCompressor)

    gassplitter_3rd_stage = neqsim.process.equipment.splitter.Splitter(
    '3rd stage anti surge splitter')
    gassplitter_3rd_stage.setInletStream(thirdStageCompressor.getOutletStream())
    gassplitter_3rd_stage.setFlowRates([-1, 1.0], "kg/hr")
    gassplitter_3rd_stage.run()
    demo_field_process.add(gassplitter_3rd_stage)

    antisurgeCalculator3 = neqsim.process.equipment.util.Calculator("anti surge calculator 3rd stage")
    antisurgeCalculator3.addInputVariable(thirdStageCompressor)
    antisurgeCalculator3.setOutputVariable(gassplitter_3rd_stage)
    antisurgeCalculator3.run()
    demo_field_process.add(antisurgeCalculator3)

    antisurgevalve3 = neqsim.process.equipment.valve.ThrottlingValve('aniti surge valve 3rd stage', gassplitter_3rd_stage.getSplitStream(1))
    antisurgevalve3.setDeltaPressure(inp.throttle_3rd_stage_recomp, "bara")
    antisurgevalve3.run()
    demo_field_process.add(antisurgevalve3)

    recyc_3rd_anti = neqsim.process.equipment.util.Recycle("recycle anti surge 3rd stage compressor")
    recyc_3rd_anti.addStream(antisurgevalve3.getOutletStream())
    recyc_3rd_anti.setOutletStream(recyclegasstream12)
    recyc_3rd_anti.setTolerance(0.001)
    recyc_3rd_anti.run()
    demo_field_process.add(recyc_3rd_anti)

    throttle_3rd_stage_recomp = neqsim.process.equipment.valve.ThrottlingValve('throttle 3rd stage recompressor', gassplitter_3rd_stage.getSplitStream(0))
    throttle_3rd_stage_recomp.setDeltaPressure(inp.throttle_3rd_stage_recomp, "bara")
    throttle_3rd_stage_recomp.run()
    demo_field_process.add(throttle_3rd_stage_recomp)

    gas2_mixer = neqsim.process.equipment.mixer.Mixer('gas mixer 2')
    gas2_mixer.addStream(throttle_3rd_stage_recomp.getOutletStream())
    gas2_mixer.addStream(gas_splitter.getSplitStream(0 ))
    gas2_mixer.addStream(test_gas_splitter.getSplitStream(0))
    gas2_mixer.run()
    demo_field_process.add(gas2_mixer)

    gas_to_injection_manifold = neqsim.process.equipment.stream.Stream('gas to injection manifold', gas2_mixer.getOutStream())
    gas_to_injection_manifold.run()
    demo_field_process.add(gas_to_injection_manifold)

    manifold = neqsim.process.equipment.manifold.Manifold('gas manifold')
    manifold.addStream(gas_to_injection_manifold)
    manifold.setSplitFactors(inp.gas_injection_manifold_split_factors)
    manifold.run()
    demo_field_process.add(manifold)

    gas_to_injection_process_A = neqsim.process.equipment.stream.Stream('gas to injection process A', manifold.getSplitStream(0))
    gas_to_injection_process_A.run()
    demo_field_process.add(gas_to_injection_process_A)

    gas_to_injection_process_B = neqsim.process.equipment.stream.Stream('gas to injection process B', manifold.getSplitStream(1))
    gas_to_injection_process_B.run()
    demo_field_process.add(gas_to_injection_process_B)

    gas_to_injection_process_C = neqsim.process.equipment.stream.Stream('gas to injection process C', manifold.getSplitStream(2))
    gas_to_injection_process_C.run()
    demo_field_process.add(gas_to_injection_process_C)

    #printFrame(gas_to_injection_manifold.getFluid())

    return demo_field_process

In [ ]:
#demo_field_main_process = createFeedProcess(inp=inp)
#demo_field_main_process.run()

#printFrame(demo_field_main_process.getUnit('gas mixer 2').getOutStream().getFluid())


# Gas injection process

In [ ]:
def gas_injection_process(inp, feedstream, demo_field_process,injection_train):
    gas_injection_process = neqsim.process.processmodel.ProcessSystem()

    if injection_train == 'train_A':
        curve_flow_1st_stage_inj = inp.curve_flow_1st_stage_inj_A
        curve_head_1st_stage_inj = inp.curve_head_1st_stage_inj_A
        curve_flow_2nd_stage_inj = inp.curve_flow_2nd_stage_inj_A
        curve_head_2nd_stage_inj = inp.curve_head_2nd_stage_inj_A
        temperature_1st_stage_injection_cooler = inp.temperature_1st_stage_injection_cooler
        pressure_1st_stage_injection_compressor = inp.pressure_1st_stage_injection_compressor
        pressuredrop_in_2ndstage_inj = inp.pressuredrop_in_2ndstage_inj
        pressuredrop_in_1ststage_inj = inp.pressuredrop_in_1ststage_inj
        pressure_2nd_stage_injection_compressor = inp.pressure_2nd_stage_injection_compressor
        temperature_2nd_stage_injection_cooler = inp.temperature_2nd_stage_injection_cooler
    elif injection_train == 'train_B':
        curve_flow_1st_stage_inj = inp.curve_flow_1st_stage_inj_B
        curve_head_1st_stage_inj = inp.curve_head_1st_stage_inj_B
        curve_flow_2nd_stage_inj = inp.curve_flow_2nd_stage_inj_B
        curve_head_2nd_stage_inj = inp.curve_head_2nd_stage_inj_B
        temperature_1st_stage_injection_cooler = inp.temperature_1st_stage_injection_cooler_B
        pressure_1st_stage_injection_compressor = inp.pressure_1st_stage_injection_compressor_B
        pressuredrop_in_2ndstage_inj = inp.pressuredrop_in_2ndstage_inj_B
        pressuredrop_in_1ststage_inj = inp.pressuredrop_in_1ststage_inj_B
        pressure_2nd_stage_injection_compressor = inp.pressure_2nd_stage_injection_compressor_B
        temperature_2nd_stage_injection_cooler = inp.temperature_2nd_stage_injection_cooler_B
    elif injection_train == 'train_C':
        curve_flow_1st_stage_inj = inp.curve_flow_1st_stage_inj_C
        curve_head_1st_stage_inj = inp.curve_head_1st_stage_inj_C
        curve_flow_2nd_stage_inj = inp.curve_flow_2nd_stage_inj_C
        curve_head_2nd_stage_inj = inp.curve_head_2nd_stage_inj_C
        temperature_1st_stage_injection_cooler = inp.temperature_1st_stage_injection_cooler_C
        pressure_1st_stage_injection_compressor = inp.pressure_1st_stage_injection_compressor_C
        pressuredrop_in_2ndstage_inj = inp.pressuredrop_in_2ndstage_inj_C
        pressuredrop_in_1ststage_inj = inp.pressuredrop_in_1ststage_inj_C
        pressure_2nd_stage_injection_compressor = inp.pressure_2nd_stage_injection_compressor_C
        temperature_2nd_stage_injection_cooler = inp.temperature_2nd_stage_injection_cooler_C
    else:
        raise ValueError("Invalid injection_train value. Choose 'train_A', 'train_B', or 'train_C'.")

    recyclegasstream_inj_1  = create_stream('recycle gas 1st stage injection')
    recyclegasstream_inj_1.setPressure(inp.first_stage_pressure-pressuredrop_in_1ststage_inj, 'bara')
    recyclegasstream_inj_1.setTemperature(25.0, 'C')
    recyclegasstream_inj_1.setFlowRate(10.0, "kg/hr")
    recyclegasstream_inj_1.run()
    gas_injection_process.add(recyclegasstream_inj_1)

    gas_mixer_inj_1 = neqsim.process.equipment.mixer.Mixer('gas mixer 1st stage injection')
    gas_mixer_inj_1.addStream(feedstream) #feedstream
    gas_mixer_inj_1.addStream(recyclegasstream_inj_1)
    gas_mixer_inj_1.run()
    gas_injection_process.add(gas_mixer_inj_1)


    firstStageInjCooler = neqsim.process.equipment.heatexchanger.Cooler(
        "1st stage cooler", gas_mixer_inj_1.getOutStream()) #change to gas_mixer_inj_1
    firstStageInjCooler.setOutTemperature(
        temperature_1st_stage_injection_cooler, 'C')
    firstStageInjCooler.run()
    gas_injection_process.add(firstStageInjCooler)

    firstStageInjScrubber1 = neqsim.process.equipment.separator.GasScrubber(
        "1st stage scrubber", firstStageInjCooler.getOutStream())
    firstStageInjScrubber1.run()
    gas_injection_process.add(firstStageInjScrubber1)

    firstStageCompressor = neqsim.process.equipment.compressor.Compressor(
        "26-KA-001", firstStageInjScrubber1.getGasOutStream())
    firstStageCompressor.setCompressorChartType("interpolate and extrapolate")
    firstStageCompressor.setUsePolytropicCalc(True)
    firstStageCompressor.setPolytropicEfficiency(0.7)
    firstStageCompressor.setOutletPressure(pressure_1st_stage_injection_compressor, 'bara')
    firstStageCompressor.getCompressorChart().getSurgeCurve().setCurve(inp.chartConditions, curve_flow_1st_stage_inj, curve_head_1st_stage_inj)
    firstStageCompressor.run()
    gas_injection_process.add(firstStageCompressor)

    gassplitter_1st_stage_inj = neqsim.process.equipment.splitter.Splitter(
    '1st stage injection anti surge splitter')
    gassplitter_1st_stage_inj.setInletStream(firstStageCompressor.getOutletStream())
    gassplitter_1st_stage_inj.setFlowRates([-1, 1.0], "kg/hr")
    gassplitter_1st_stage_inj.run()
    gas_injection_process.add(gassplitter_1st_stage_inj)

    antisurgeCalculator_1_inj = neqsim.process.equipment.util.Calculator("anti surge calculator_1")
    antisurgeCalculator_1_inj.addInputVariable(firstStageCompressor)
    antisurgeCalculator_1_inj.setOutputVariable(gassplitter_1st_stage_inj)
    antisurgeCalculator_1_inj.run()
    gas_injection_process.add(antisurgeCalculator_1_inj)

    antisurgevalve_1_inj = neqsim.process.equipment.valve.ThrottlingValve('aniti surge valve 1st stage injection', gassplitter_1st_stage_inj.getSplitStream(1))
    antisurgevalve_1_inj.setOutletPressure(inp.first_stage_pressure-pressuredrop_in_1ststage_inj, "bara") #add input pressuredrop_in_1ststage_inj as above
    antisurgevalve_1_inj.run()
    gas_injection_process.add(antisurgevalve_1_inj)

    recycl1 = neqsim.process.equipment.util.Recycle("recycle anti surge 1st stage injection compressor")
    recycl1.addStream(antisurgevalve_1_inj.getOutletStream())
    recycl1.setOutletStream(recyclegasstream_inj_1)
    recycl1.setTolerance(0.001)
    recycl1.run()
    gas_injection_process.add(recycl1) ###

    ## second stage injection

    recyclegasstream_inj_2  = create_stream('recycle gas 2nd stage injection')
    recyclegasstream_inj_2.setPressure(pressure_1st_stage_injection_compressor-pressuredrop_in_2ndstage_inj, 'bara')
    recyclegasstream_inj_2.setTemperature(25.0, 'C')
    recyclegasstream_inj_2.setFlowRate(10.0, "kg/hr")
    recyclegasstream_inj_2.run()
    gas_injection_process.add(recyclegasstream_inj_2)

    gas_mixer_inj_2 = neqsim.process.equipment.mixer.Mixer('gas mixer 2nd stage injection')
    gas_mixer_inj_2.addStream(gassplitter_1st_stage_inj.getSplitStream(0)) #change to gassplitter_1st_stage_inj
    gas_mixer_inj_2.addStream(recyclegasstream_inj_2)
    gas_mixer_inj_2.run()
    gas_injection_process.add(gas_mixer_inj_2)

    seccondStageInjCooler = neqsim.process.equipment.heatexchanger.Cooler(
        "2nd stage cooler",  gas_mixer_inj_2.getOutStream())
    seccondStageInjCooler.setOutTemperature(
        temperature_2nd_stage_injection_cooler, 'C')
    seccondStageInjCooler.run()
    gas_injection_process.add(seccondStageInjCooler)

    secondStageInjScrubber1 = neqsim.process.equipment.separator.GasScrubber(
        "2nd stage scrubber", seccondStageInjCooler.getOutStream())
    secondStageInjScrubber1.run()
    gas_injection_process.add(secondStageInjScrubber1)

    mixer_liquid_from_scrubber = neqsim.process.equipment.mixer.Mixer('mixer liquid from scrubber')
    mixer_liquid_from_scrubber.addStream(firstStageInjScrubber1.getLiquidOutStream())
    mixer_liquid_from_scrubber.addStream(secondStageInjScrubber1.getLiquidOutStream())
    mixer_liquid_from_scrubber.run()
    gas_injection_process.add(mixer_liquid_from_scrubber)

    liquid_return= neqsim.process.equipment.stream.Stream('liquid return', mixer_liquid_from_scrubber.getOutStream())
    liquid_return.run()
    gas_injection_process.add(liquid_return)

    # Recycle liquid back to 2nd stage separator using proper recycle unit
    liquid_recycle = neqsim.process.equipment.util.Recycle("liquid return recycle")
    liquid_recycle.addStream(liquid_return)
    if injection_train == 'train_A':
        liquid_recycle.setOutletStream(demo_field_process.getUnit('recycle 2nd stage A'))
    elif injection_train == 'train_B':
        liquid_recycle.setOutletStream(demo_field_process.getUnit('recycle 2nd stage B'))
    elif injection_train == 'train_C':
        liquid_recycle.setOutletStream(demo_field_process.getUnit('recycle 2nd stage C'))

    liquid_recycle.setTolerance(1e-2)
    liquid_recycle.run()
    gas_injection_process.add(liquid_recycle)

    secondStageCompressor = neqsim.process.equipment.compressor.Compressor(
        "26-KA-002", secondStageInjScrubber1.getGasOutStream())
    secondStageCompressor.setCompressorChartType("interpolate and extrapolate")
    secondStageCompressor.setUsePolytropicCalc(True)
    secondStageCompressor.setPolytropicEfficiency(0.7)
    secondStageCompressor.setOutletPressure(pressure_2nd_stage_injection_compressor, 'bara')
    secondStageCompressor.getCompressorChart().getSurgeCurve().setCurve(inp.chartConditions, curve_flow_2nd_stage_inj, curve_head_2nd_stage_inj)
    secondStageCompressor.run()
    gas_injection_process.add(secondStageCompressor)

    gassplitter_2nd_stage_inj = neqsim.process.equipment.splitter.Splitter(
    '2nd stage injection anti surge splitter')
    gassplitter_2nd_stage_inj.setInletStream(secondStageCompressor.getOutletStream())
    gassplitter_2nd_stage_inj.setFlowRates([-1, 1.0], "kg/hr")
    gassplitter_2nd_stage_inj.run()
    gas_injection_process.add(gassplitter_2nd_stage_inj)

    antisurgeCalculator_2_inj = neqsim.process.equipment.util.Calculator("anti surge calculator_2")
    antisurgeCalculator_2_inj.addInputVariable(secondStageCompressor)
    antisurgeCalculator_2_inj.setOutputVariable(gassplitter_2nd_stage_inj)
    antisurgeCalculator_2_inj.run()
    gas_injection_process.add(antisurgeCalculator_2_inj)

    antisurgevalve_2_inj = neqsim.process.equipment.valve.ThrottlingValve('aniti surge valve 2nd stage injection', gassplitter_2nd_stage_inj.getSplitStream(1))
    antisurgevalve_2_inj.setOutletPressure(pressure_1st_stage_injection_compressor-pressuredrop_in_2ndstage_inj, "bara") # as above
    antisurgevalve_2_inj.run()
    gas_injection_process.add(antisurgevalve_2_inj)

    recycl2 = neqsim.process.equipment.util.Recycle("recycle anti surge 2nd stage injection compressor")
    recycl2.addStream(antisurgevalve_2_inj.getOutletStream())
    recycl2.setOutletStream(recyclegasstream_inj_2)
    recycl2.setTolerance(0.001)
    recycl2.run()
    gas_injection_process.add(recycl2) ###

    gas_to_injection = neqsim.process.equipment.stream.Stream('gas to injection', gassplitter_2nd_stage_inj.getSplitStream(0))
    gas_to_injection.run()
    gas_injection_process.add(gas_to_injection)

    return gas_injection_process

# Full process

In [ ]:
demo_field_main_process = createFeedProcess(inp=inp)
demo_field_main_process.run()

demo_field_injection_A = gas_injection_process(inp, demo_field_main_process.getUnit('gas to injection process A'), demo_field_main_process, 'train_A')
demo_field_injection_A.run()

demo_field_injection_B = gas_injection_process(inp, demo_field_main_process.getUnit('gas to injection process B'), demo_field_main_process, 'train_B')
demo_field_injection_B.run()

demo_field_injection_C = gas_injection_process(inp, demo_field_main_process.getUnit('gas to injection process C'), demo_field_main_process, 'train_C')
demo_field_injection_C.run()


In [ ]:
# Check gas flow rates to injection
#demo_field_main_process.getUnit('gas to injection process B').getFlowRate('m3/hr')
#demo_field_main_process.getUnit('gas to injection process A').getFlowRate('m3/hr')
#demo_field_main_process.getUnit('gas to injection process C').getFlowRate('m3/hr')

In [ ]:
demo_field_process = neqsim.process.processmodel.ProcessModel()
demo_field_process.add("main process", demo_field_main_process)
demo_field_process.add("injection A",demo_field_injection_A)
demo_field_process.add("injection B",demo_field_injection_B)
demo_field_process.add("injection C",demo_field_injection_C)
#demo_field_process.setRunStep(True)
demo_field_process.run()

# Valve checks

In [ ]:
#demo_field_process.get('main process').getUnit('aniti surge valve').getInletStream().setFlowRate(5.0, "kg/hr")
#demo_field_process.get('main process').getUnit('aniti surge valve').getInletStream().run()
#demo_field_process.get('main process').getUnit('aniti surge valve').run()
#demo_field_process.get('main process').getUnit('aniti surge valve').calcKv()
#print('Cv ', demo_field_process.get('main process').getUnit('aniti surge valve').getCv('US'))

demo_field_process.get('main process').getUnit('aniti surge valve').calcKv()
print('Cv ', demo_field_process.get('main process').getUnit('aniti surge valve').getCv('US'))

print('Cv ', demo_field_process.get('main process').getUnit('liquid control valve first stage').getCv('US'))
print('flow rate ', demo_field_process.get('main process').getUnit('liquid control valve first stage').getInletStream().getFlowRate('kg/hr'))

print('Cv ', demo_field_process.get('main process').getUnit('liquid control valve second stage').getCv('US'))
print('flow rate ', demo_field_process.get('main process').getUnit('liquid control valve second stage').getInletStream().getFlowRate('kg/hr'))

In [ ]:
from neqsim.thermo import TPflash
fluidcopy = demo_field_process.get('main process').getUnit('VG-23-006').getFeedStream().getFluid().clone()
#fluidcopy.setPressure(4.275, "bara" )
TPflash(fluidcopy)

print(list(fluidcopy.getMolarComposition()))

printFrame(fluidcopy)

# Mass balance check

In [ ]:
feed = demo_field_process.get('main process').getUnit('feed mixer 2nd').getOutStream().getFlowRate('kg/hr') + demo_field_process.get('main process').getUnit('Test separator').getFeedStream().getFlowRate('kg/hr') + demo_field_process.get('main process').getUnit('1st stage separator').getFeedStream().getFlowRate('kg/hr')+ demo_field_process.get('injection A').getUnit('liquid return').getFlowRate('kg/hr') + demo_field_process.get('injection B').getUnit('liquid return').getFlowRate('kg/hr') + demo_field_process.get('injection C').getUnit('liquid return').getFlowRate('kg/hr')

print('feed ', feed)

outlet = demo_field_process.get('main process').getUnit('Test separator').getWaterOutStream().getFlowRate('kg/hr') + demo_field_process.get('main process').getUnit('1st stage separator').getWaterOutStream().getFlowRate('kg/hr')+ demo_field_process.get('main process').getUnit('2nd stage separator').getWaterOutStream().getFlowRate('kg/hr')+ demo_field_process.get('main process').getUnit('3RD stage separator').getOilOutStream().getFlowRate('kg/hr')+ demo_field_process.get('main process').getUnit('gas manifold').getSplitStream(0).getFlowRate('kg/hr')+ demo_field_process.get('main process').getUnit('gas manifold').getSplitStream(1).getFlowRate('kg/hr')+ demo_field_process.get('main process').getUnit('gas manifold').getSplitStream(2).getFlowRate('kg/hr')
+ demo_field_process.get('main process').getUnit('stream to closed drain').getFlowRate('kg/hr')
print('outlet ',  outlet)

print('mass balance ', (outlet-feed)/feed*100 , ' %')

# Export results

In [ ]:
from neqsim.process import results_json

year = inp.Year
output_file_path = f"results/demo_field_{year}.json"

try:
    output_demo_field_process = results_json(demo_field_process, output_file_path)
except :
    print('error in demo_field process json export')

#clear_output(wait=True)

print(f"\nOutput saved to {output_file_path}")

# Visualize process flow diagram

In [ ]:
import graphviz

options = neqsim.process.processmodel.ProcessSystemGraphvizExporter.GraphvizExportOptions.builder().includeStreamTemperatures(True).includeStreamPressures(True).includeStreamFlowRates(True).includeStreamPropertyTable(False).tablePlacement(
                neqsim.process.processmodel.ProcessSystemGraphvizExporter.GraphvizExportOptions.TablePlacement.BELOW).build()

demo_field_process.get('main process').exportToGraphviz("demo_field_process.dot", options)
demo_field_process.get('injection A').exportToGraphviz("demo_field_injA_process.dot", options)
demo_field_process.get('injection B').exportToGraphviz("demo_field_injB_process.dot", options)
demo_field_process.get('injection C').exportToGraphviz("demo_field_injC_process.dot", options)

In [ ]:
# Mass balance check for all init operations in demo_field_process
print("=" * 80)
print("MASS BALANCE CHECK FOR INIT OPERATIONS")
print("=" * 80)

# Check mass balance for each process module
# Returns: Map<String, Map<String, ProcessSystem.MassBalanceResult>>
# Structure: {process_name: {unit_name: MassBalanceResult}}
all_mass_balance_results = demo_field_process.checkMassBalance("kg/hr")

# Display results for each process
total_units = 0
total_failed = 0

for process_name, unit_results in all_mass_balance_results.items():
    print(f"\n{process_name}:")
    print("-" * 80)

    if not unit_results:
        print("  No unit operations found")
        continue

    failed_count = 0
    for unit_name, result in unit_results.items():
        total_units += 1
        abs_error = result.getAbsoluteError()
        percent_error = result.getPercentError()

        # Check if unit failed (>0.1% error)
        if abs(percent_error) > 0.1:
            failed_count += 1
            total_failed += 1
            status = "✗"
        else:
            status = "✓"

        print(f"  {status} {unit_name}: {abs_error:.6f} {result.getUnit()} ({percent_error:.4f}%)")

    if failed_count > 0:
        print(f"  → {failed_count} unit(s) failed in this process")

# Check for failed units across all processes
print("\n" + "=" * 80)
print("FAILED MASS BALANCE UNITS (>0.1% error)")
print("=" * 80)

failed_results = demo_field_process.getFailedMassBalance("kg/hr", 0.1)

if not failed_results:
    print("\n✓ All processes and units passed mass balance check!")
else:
    print(f"\n✗ Found {total_failed} unit(s) with mass balance errors:\n")
    for process_name, failed_units in failed_results.items():
        print(f"\n  Process: {process_name}")
        for unit_name, result in failed_units.items():
            print(f"    - {unit_name}:")
            print(f"      Absolute Error: {result.getAbsoluteError():.6f} {result.getUnit()}")
            print(f"      Percent Error: {result.getPercentError():.4f}%")

print("\n" + "=" * 80)
print(f"SUMMARY: {total_units} total units checked, {total_failed} failed")
print("=" * 80)